# Fe & As concentrations in Flemish Aquifersystems

In this more complex study case pydov was used to query groundwater filter and groundwater sample data. 
The goal was to find the iron (Fe) and arsenic (As) concentrations for each groundwater filter in monitoring networks 1 and 8, searing in each main Hydrogeological Coding of the Subsurface of Flanders (HCOV) unit. The maximum concentration in each filter over the entire measurement period in the database was then mapped per HCOV unit and subdivided into specific categories. The reporting limit, the drinking water standard and the environmental quality standard of As and Fe were taken into account when choosing these categories. This was in an attempt to make a kind of groundwater quality map for Fe and As in Flanders. 

## For HCOV 0100: Quaternary Aquifersystem

In [31]:
import pydov
from pydov.search.grondwaterfilter import GrondwaterFilterSearch
from pydov.search.grondwatermonster import GrondwaterMonsterSearch
import pandas as pd
pydov.request_timeout = 300

gwfilter = GrondwaterFilterSearch()
gwmonster = GrondwaterMonsterSearch()

fields_filter = gwfilter.get_fields()
fields_monster = gwmonster.get_fields()

In [32]:
from owslib.fes import Or, And, PropertyIsEqualTo,  PropertyIsLike

# For HCOV unit 0100:
query_filter = And([Or([PropertyIsLike(propertyname='meetnet', literal='meetnet 1 %'),
                       PropertyIsLike(propertyname='meetnet', literal='meetnet 8 %')]),
                    PropertyIsLike(propertyname='aquifer', literal='0100 - %')])

df_filters = gwfilter.search(query=query_filter,
                     return_fields=('pkey_filter', 'meetnet', 'x', 'y', 'aquifer', 'aquifer_code', 'grondwaterlichaam_code', 'lengte_filter'))

# first Or([]) is an alternative to Join
# second Or([]) is a specification to speed up querying, only interested in samples that actually measured Fe (a cation) and As (a heavy metal)
query_monsters =  And([Or([PropertyIsEqualTo(propertyname='pkey_filter', literal=pkey) for pkey in df_filters['pkey_filter']]),
                      Or([PropertyIsEqualTo(propertyname='kationen', literal='True'), PropertyIsEqualTo(propertyname='zware_metalen', literal='True')])])

df_monsters = gwmonster.search(query=query_monsters,
                                   return_fields=('pkey_filter', 'datum_monstername', 'x', 'y', 'parameter', 'waarde', 'eenheid'),
                               max_features=300) # too many features in the first HCOV to query, so we look at a small sample first

df_monsters['aquifer'] = '0100' # Add the HCOV to the df

# For Fe
df_Fe = df_monsters.loc[df_monsters['parameter'] == ('Fe')]
# df_Fe.to_csv('Fe_HCOV_0100.csv')
df_Fe.head()

[000/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[200/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[250/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[300/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[350/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[400/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[450/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[500/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[550/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[600/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[650/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[700/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[750/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[800/188

,pkey_filter,datum_monstername,x,y,parameter,waarde,eenheid,aquifer
1,https://www.dov.vlaanderen.be/data/filter/1993...,2007-07-17,148333.0,191187.0,Fe,3.9800,mg/l,0100
38,https://www.dov.vlaanderen.be/data/filter/2003...,2009-02-27,30548.0,199800.0,Fe,3.7200,mg/l,0100
90,https://www.dov.vlaanderen.be/data/filter/2003...,2006-02-21,30548.0,199800.0,Fe,1.6200,mg/l,0100
106,https://www.dov.vlaanderen.be/data/filter/1981...,2016-06-21,192776.0,213392.0,Fe,13.1521,mg/l,0100
161,https://www.dov.vlaanderen.be/data/filter/2003...,2010-02-24,30548.0,199800.0,Fe,0.0800,mg/l,0100


In [33]:
# statistics on concentration per filter
df_Fe_stat = df_Fe.groupby(['pkey_filter', 'x', 'y', 'parameter', 'eenheid']).describe().waarde
# df_Fe_stat.to_csv('Fe_HCOV_0100_stat.csv')
df_Fe_stat.head()

,,,,,count,mean,std,min,25%,50%,75%,max
pkey_filter,x,y,parameter,eenheid,,,,,,,,
https://www.dov.vlaanderen.be/data/filter/1968-001009,100443.00000,208918.0,Fe,mg/l,3.0,2.506667,1.375403,0.9200,2.0800,3.2400,3.3000,3.3600
https://www.dov.vlaanderen.be/data/filter/1979-001105,243510.00000,197585.0,Fe,mg/l,2.0,3.360000,4.582052,0.1200,1.7400,3.3600,4.9800,6.6000
https://www.dov.vlaanderen.be/data/filter/1979-001106,245905.46875,197991.5,Fe,mg/l,3.0,10.383333,8.242623,1.1500,7.0750,13.0000,15.0000,17.0000
https://www.dov.vlaanderen.be/data/filter/1979-001108,235500.00000,185005.0,Fe,mg/l,2.0,0.020000,0.000000,0.0200,0.0200,0.0200,0.0200,0.0200
https://www.dov.vlaanderen.be/data/filter/1981-006408,192776.00000,213392.0,Fe,mg/l,1.0,13.152100,NaN,13.1521,13.1521,13.1521,13.1521,13.1521


In [34]:
# For As
df_As = df_monsters.loc[df_monsters['parameter'] == ('As')]
# df_As.to_csv('As_HCOV_0100.csv')
df_As.head()

,pkey_filter,datum_monstername,x,y,parameter,waarde,eenheid,aquifer
3,https://www.dov.vlaanderen.be/data/filter/1993...,2007-07-17,148333.0,191187.0,As,5.0000,µg/l,0100
54,https://www.dov.vlaanderen.be/data/filter/2003...,2009-02-27,30548.0,199800.0,As,15.0000,µg/l,0100
98,https://www.dov.vlaanderen.be/data/filter/2003...,2006-02-21,30548.0,199800.0,As,5.0000,µg/l,0100
104,https://www.dov.vlaanderen.be/data/filter/1981...,2016-06-21,192776.0,213392.0,As,24.0142,µg/l,0100
166,https://www.dov.vlaanderen.be/data/filter/2003...,2010-02-24,30548.0,199800.0,As,5.0000,µg/l,0100


In [35]:
# statistics on concentration per filter
df_As_stat = df_As.groupby(['pkey_filter', 'x', 'y', 'parameter', 'eenheid']).describe().waarde
# df_As_stat.to_csv('As_HCOV_0100_stat.csv')
df_As_stat.head()

,,,,,count,mean,std,min,25%,50%,75%,max
pkey_filter,x,y,parameter,eenheid,,,,,,,,
https://www.dov.vlaanderen.be/data/filter/1968-001009,100443.00000,208918.0,As,µg/l,3.0,8.786333,2.995714,6.7000,7.0700,7.4400,9.8295,12.2190
https://www.dov.vlaanderen.be/data/filter/1979-001105,243510.00000,197585.0,As,µg/l,2.0,100.000000,140.007143,1.0000,50.5000,100.0000,149.5000,199.0000
https://www.dov.vlaanderen.be/data/filter/1979-001106,245905.46875,197991.5,As,µg/l,3.0,5.566667,3.007214,3.4000,3.8500,4.3000,6.6500,9.0000
https://www.dov.vlaanderen.be/data/filter/1979-001108,235500.00000,185005.0,As,µg/l,2.0,5.000000,0.000000,5.0000,5.0000,5.0000,5.0000,5.0000
https://www.dov.vlaanderen.be/data/filter/1981-006408,192776.00000,213392.0,As,µg/l,1.0,24.014200,NaN,24.0142,24.0142,24.0142,24.0142,24.0142


Due to the fact that there are too many features in HCOV 0100 that we want to query (>10 000), Flanders was split down the middle into two bounding boxes (still not all data is included in the search => max_features=9999, but a few more data points were added by this method): 

In [36]:
from owslib.fes import Or, And, PropertyIsEqualTo,  PropertyIsLike
from pydov.util.location import Within, Box

# West half of Flanders
df_filter_0100 = gwfilter.search(location=Within(Box(0, 150000, 150000, 250000)),
                             query=query_filter,
                             return_fields=('pkey_filter', 'meetnet', 'x', 'y', 'aquifer', 'aquifer_code', 'grondwaterlichaam_code', 'lengte_filter'))

query_monsters = And([Or([PropertyIsEqualTo(propertyname='pkey_filter', literal=pkey) for pkey in df_filter_0100['pkey_filter']]),
                     Or([PropertyIsEqualTo(propertyname='kationen', literal='True'), PropertyIsEqualTo(propertyname='zware_metalen', literal='True')])])

df_monsters = gwmonster.search(query=query_monsters,
                               return_fields=('pkey_filter', 'datum_monstername', 'x', 'y', 'parameter', 'waarde', 'eenheid'),
                               max_features=9999)

df_monsters['aquifer'] = '0100'

# For Fe
df_Fe = df_monsters.loc[df_monsters['parameter'] == ('Fe')]
# df_Fe.to_csv('Fe_HCOV_0100_west.csv')

# statistics on concentration per filter
df_Fe_stat = df_Fe.groupby(['pkey_filter', 'x', 'y', 'parameter', 'eenheid']).describe().waarde
# df_Fe_stat.to_csv('Fe_HCOV_0100_stat.csv')

# For As
df_As = df_monsters.loc[df_monsters['parameter'] == ('As')]
# df_As.to_csv('As_HCOV_0100.csv')

# statistics on concentration per filter
df_As_stat = df_As.groupby(['pkey_filter', 'x', 'y', 'parameter', 'eenheid']).describe().waarde
# df_As_stat.to_csv('As_HCOV_0100_stat.csv')

[000/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[200/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[250/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[300/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[350/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[400/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[450/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[500/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[550/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[600/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[650/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[700/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[750/1484] cccccccccccccccccccccccccccccccccccccccccccccccccc
[800/148

In [37]:
# East half of Flanders 

df_filters = gwfilter.search(location=Within(Box(150000, 150000, 250000, 250000)),
                             query=query_filter,
                             return_fields=('pkey_filter', 'meetnet', 'x', 'y', 'aquifer', 'aquifer_code', 'grondwaterlichaam_code', 'lengte_filter'))

query_monsters = And([Or([PropertyIsEqualTo(propertyname='pkey_filter', literal=pkey) for pkey in df_filter_0100['pkey_filter']]),
                     Or([PropertyIsEqualTo(propertyname='kationen', literal='True'), PropertyIsEqualTo(propertyname='zware_metalen', literal='True')])])

df_monsters = gwmonster.search(query=query_monsters,
                               return_fields=('pkey_filter', 'datum_monstername', 'x', 'y', 'parameter', 'waarde', 'eenheid'),
                               max_features=9999)

df_monsters['aquifer'] = '0100' # Geef HCOV mee aan df
df_monsters.head()

[000/399] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/399] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/399] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/399] cccccccccccccccccccccccccccccccccccccccccccccccccc
[200/399] cccccccccccccccccccccccccccccccccccccccccccccccccc
[250/399] cccccccccccccccccccccccccccccccccccccccccccccccccc
[300/399] cccccccccccccccccccccccccccccccccccccccccccccccccc
[350/399] ccccccccccccccccccccccccccccccccccccccccccccccccc
[000/9999] cccccccccccccccccccccccccccccccccccccccccccccccccc (53 min. left)
[050/9999] cccccccccccccccccccccccccccccccccccccccccccccccccc (27 min. left)
[100/9999] cccccccccccccccccccccccccccccccccccccccccccccccccc (18 min. left)
[150/9999] cccccccccccccccccccccccccccccccccccccccccccccccccc (13 min. left)
[200/9999] cccccccccccccccccccccccccccccccccccccccccccccccccc (10 min. left)
[250/9999] cccccccccccccccccccccccccccccccccccccccccccccccccc (9 min. left)
[300/9999] ccccccccccccccccccccccccccccccccccccccccc

,pkey_filter,datum_monstername,x,y,parameter,waarde,eenheid,aquifer
0,https://www.dov.vlaanderen.be/data/filter/1993...,2007-07-17,148333.0,191187.0,NO3,0.31,mg/l,0100
1,https://www.dov.vlaanderen.be/data/filter/1993...,2007-07-17,148333.0,191187.0,Fe,3.98,mg/l,0100
2,https://www.dov.vlaanderen.be/data/filter/1993...,2007-07-17,148333.0,191187.0,Al,0.05,mg/l,0100
3,https://www.dov.vlaanderen.be/data/filter/1993...,2007-07-17,148333.0,191187.0,As,5.00,µg/l,0100
4,https://www.dov.vlaanderen.be/data/filter/1993...,2007-07-17,148333.0,191187.0,pH,6.70,Sörensen,0100


In [38]:
# For Fe
df_Fe = df_monsters.loc[df_monsters['parameter'] == ('Fe')]
# df_Fe.to_csv('Fe_HCOV_0100_west.csv')

# statistics on concentration per filter
df_Fe_stat = df_Fe.groupby(['pkey_filter', 'x', 'y', 'parameter', 'eenheid']).describe().waarde
# df_Fe_stat.to_csv('Fe_HCOV_0100_stat.csv')

# For As
df_As = df_monsters.loc[df_monsters['parameter'] == ('As')]
# df_As.to_csv('As_HCOV_0100.csv')

# statistics on concentration per filter
df_As_stat = df_As.groupby(['pkey_filter', 'x', 'y', 'parameter', 'eenheid']).describe().waarde
# df_As_stat.to_csv('As_HCOV_0100_stat.csv')

## Other HCOV units
For the rest of the HCOV units a "for" loop was used, otherwise the amount of features surely exceeds 10 000. 
To use the data in QGIS, it must be saved to csv files (in hashtags). These can then be loaded in QGIS as layers containing points which have the statistics on the concentrations of Fe & As. 

In [39]:
# for each subsequent main HCOV unit + 0230, 0252 & 0254
hcov_eenheden = ['0200', '0300', '0400', '0500', '0600', '0700', '0800', '0900', '1000', '1100', '1300',
                 '0230', '0252', '0254']        # HCOV 1200 doesn't have measurements 

for HCOV in hcov_eenheden:
    query_hcov = And([Or([PropertyIsLike(propertyname='meetnet', literal='meetnet 1 %'),
                          PropertyIsLike(propertyname='meetnet', literal='meetnet 8 %')]),
                      PropertyIsLike(propertyname='aquifer', literal='{} - %'.format(HCOV))])

    df_filter_hcov = gwfilter.search(query=query_hcov,
                                     return_fields=('pkey_filter', 'meetnet', 'x', 'y', 'aquifer', 'aquifer_code', 'grondwaterlichaam_code', 'lengte_filter'))

    query_monster_hcov =  And([Or([PropertyIsEqualTo(propertyname='pkey_filter', literal=pkey) for pkey in df_filter_hcov['pkey_filter']]),
                      Or([PropertyIsEqualTo(propertyname='kationen', literal='True'), PropertyIsEqualTo(propertyname='zware_metalen', literal='True')])])

    df_monster_hcov = gwmonster.search(query=query_monster_hcov,
                                       return_fields=('pkey_filter', 'datum_monstername', 'x', 'y', 'parameter', 'waarde', 'eenheid'),
                                       max_features=9999)   # the maximum cannot be exceeded
    df_monster_hcov['aquifer'] = HCOV           # add the HCOV to the samples
    # df_monster_hcov.to_csv('HCOV_{}_monsters.csv'.format(HCOV))

    # Save measurements for Fe in HCOV 
    df_Fe = df_monster_hcov.loc[df_monster_hcov['parameter'] == ('Fe')]
    # df_Fe.to_csv('Fe_HCOV_{}.csv'.format(HCOV))
    df_Fe_stat = df_Fe.groupby(['pkey_filter', 'x', 'y', 'parameter', 'eenheid']).describe().waarde
    df_Fe_stat.head()
    # df_Fe_stat.to_csv('Fe_HCOV_{}_stat.csv'.format(HCOV))

    # Save measurements for As in HCOV
    df_As = df_monster_hcov.loc[df_monster_hcov['parameter'] == ('As')]
    # df_As.to_csv('As_HCOV_{}.csv'.format(HCOV))
    df_As_stat = df_As.groupby(['pkey_filter', 'x', 'y', 'parameter', 'eenheid']).describe().waarde
    df_As_stat.head()
    # df_As_stat.to_csv('As_HCOV_{}_stat.csv'.format(HCOV))


[000/184] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/184] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/184] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/184] cccccccccccccccccccccccccccccccccc
[000/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc (6 min. left)
[050/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc (3 min. left)
[100/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc (2 min. left)
[150/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc
[200/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc
[250/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc
[300/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc
[350/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc
[400/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc
[450/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc
[500/4879] cccccccccccccccccccccccccccccccccccccccccccccccccc
[550/4879] ccccccccccccccccccccccccccccccccccccc

## QGIS application


With the csv files and the x and y coördinates in them, the filters (with statistics on parameter values, including max concentration) can be added to QGIS as points. The coördinate system is Lambert72 (EPSG:31370) (standard for DOV). 
The maps of the occurence of HCOVs are available from DOV via WCS (https://www.dov.vlaanderen.be/geoserver/wcs?) and can also easily be imported in QGIS. 

### Fe in HCOV 0100
<img src="Images/HCOV_0100_Fe.PNG" />

These points were categorized based on the reporting limit, the drinking water standard and the environmental quality standard of Fe. Quite a lot of areas exceed the environmental quality standard, though these points are quite scattered. 

<img src="Images/HCOV_0100_Fe_categorized.PNG" />

### As in HCOV 0100

The same principle was used for As, with its own categories. 

<img src="Images/HCOV_0100_Fe_categorized.PNG" />





#### Notes: 
The person reading this should take into consideration the limitations of this data. With this code you query data per HCOV unit code, and each measurement has only one unit code assigned. Here only the main units have been queried (eg HCOV 0100), but that does not include the measurements with a code within the subdivisions of that main unit (such as HCOV 0110, 0120, 130 or even 0131, 0132,...). To be complete, you should therefore explicitly request all existing subdivisions (HCOV codes) and add them to the data of each main unit. (a few elifs are necessary for this, to check whether there are filters where samples have been taken within this HCOV, we know there are none in HCOV 1200 for example). A list of the whole HCOV is given here: 

In [40]:
hcov_eenheden = ['0100', '0110', '0120', '0130', '0131', '0132', '0133', '0134','0135', '0140', '0150', '0151', '0152', '0153', '0154', '0160', '0161', '0162', '0163', '0170', '0171', '0172', '0173',
                 '0200', '0210', '0211', '0212', '0213', '0214', '0215', '0220', '0221', '0222', '0223','0230', '0231', '0232', '0233', '0234', '0240', '0241', '0242', '0250', '0251', '0252', '0253', '0254', '0255', '0256',
                 '0300', '0301', '0302', '0303', '0304',
                 '0400', '0410', '0420', '0430', '0431', '0432', '0433', '0434', '0435', '0436',
                 '0440', '0441', '0442', '0443', '0450', '0451', '0452', '0453',
                 '0500', '0501', '0502', '0503', '0504', '0505',
                 '0600', '0610', '0611', '0612', '0620', '0630', '0631', '0632', '0640',
                 '0700', '0701', '0702', '0800', '0900', '0910', '0920', '0921', '0922', '0923','0924', '0925',
                 '1000', '1010', '1011', '1012', '1013', '1014', '1015', '0120', '0121', '0122', '0123',
                 '1030', '1031', '1032', '1033', '1034', '1035',
                 '1100', '1110', '1111', '1112', '1113', '1120', '1130', '1140', '1150',
                 '1200', '1210', '1220', '1230',
                 '1300', '1310', '1320', '1330', '1340']

In [42]:
# check pydov
import pydov
from pydov.search.grondwaterfilter import GrondwaterFilterSearch
from pydov.search.grondwatermonster import GrondwaterMonsterSearch
import pandas as pd
pydov.request_timeout = 300

gwfilter = GrondwaterFilterSearch()
gwmonster = GrondwaterMonsterSearch()

fields_filter = gwfilter.get_fields()
fields_monster = gwmonster.get_fields()

from owslib.fes import Or, And, PropertyIsEqualTo,  PropertyIsLike

# For all HCOV units
hcov_eenheden = ['0100', '0110', '0120', '0130', '0131', '0132', '0133', '0134','0135', '0140', '0150', '0151', '0152', '0153', '0154', '0160', '0161', '0162', '0163', '0170', '0171', '0172', '0173',
                 '0200', '0210', '0211', '0212', '0213', '0214', '0215', '0220', '0221', '0222', '0223','0230', '0231', '0232', '0233', '0234', '0240', '0241', '0242', '0250', '0251', '0252', '0253', '0254', '0255', '0256',
                 '0300', '0301', '0302', '0303', '0304',
                 '0400', '0410', '0420', '0430', '0431', '0432', '0433', '0434', '0435', '0436',
                 '0440', '0441', '0442', '0443', '0450', '0451', '0452', '0453',
                 '0500', '0501', '0502', '0503', '0504', '0505',
                 '0600', '0610', '0611', '0612', '0620', '0630', '0631', '0632', '0640',
                 '0700', '0701', '0702', '0800', '0900', '0910', '0920', '0921', '0922', '0923','0924', '0925',
                 '1000', '1010', '1011', '1012', '1013', '1014', '1015', '0120', '0121', '0122', '0123',
                 '1030', '1031', '1032', '1033', '1034', '1035',
                 '1100', '1110', '1111', '1112', '1113', '1120', '1130', '1140', '1150',
                 '1200', '1210', '1220', '1230',
                 '1300', '1310', '1320', '1330', '1340']

empty_hcov = list()
df_monsters = pd.DataFrame(columns=('pkey_filter', 'datum_monstername', 'x', 'y', 'parameter', 'waarde', 'eenheid'))

for HCOV in hcov_eenheden:

    query_hcov = And([Or([PropertyIsLike(propertyname='meetnet', literal='meetnet 1 %'),
                          PropertyIsLike(propertyname='meetnet', literal='meetnet 8 %')]),
                      PropertyIsLike(propertyname='aquifer', literal='{} - %'.format(HCOV))])

    df_filter_hcov = gwfilter.search(query=query_hcov,
                                     return_fields=('pkey_filter', 'meetnet', 'x', 'y', 'aquifer', 'aquifer_code', 'grondwaterlichaam_code', 'lengte_filter'))
    if len(df_filter_hcov) >= 2:        # (more than) 2 filters

        query_monster_hcov =  And([Or([PropertyIsEqualTo(propertyname='pkey_filter', literal=pkey) for pkey in df_filter_hcov['pkey_filter']]),
                          Or([PropertyIsEqualTo(propertyname='kationen', literal='True'), PropertyIsEqualTo(propertyname='zware_metalen', literal='True')])])

        df_monster_hcov = gwmonster.search(query=query_monster_hcov,
                                           return_fields=('pkey_filter', 'datum_monstername', 'x', 'y', 'parameter', 'waarde', 'eenheid'),
                                           max_features=9999)   
        df_monster_hcov['aquifer'] = HCOV           # add HCOV
        # df_monster_hcov.to_csv('HCOV_{}_monsters.csv'.format(HCOV))

        # For Fe in the HCOV
        df_Fe = df_monster_hcov.loc[df_monster_hcov['parameter'] == ('Fe')]
        # df_Fe.to_csv('Fe_HCOV_{}.csv'.format(HCOV))
        
        # queried samples containing measurements of cations OR heavy metals, if one of either Fe/As is not present it could be an empty DataFrame
        if not df_Fe.empty:
            df_Fe_stat = df_Fe.groupby(['pkey_filter', 'x', 'y']).describe().waarde
            # df_Fe_stat.to_csv('Fe_HCOV_{}_stat.csv'.format(HCOV))

        # For As in the HCOV
        df_As = df_monster_hcov.loc[df_monster_hcov['parameter'] == ('As')]
        # df_As.to_csv('As_HCOV_{}.csv'.format(HCOV))
        if not df_As.empty:
            df_As_stat = df_As.groupby(['pkey_filter', 'x', 'y']).describe().waarde
            # df_As_stat.to_csv('As_HCOV_{}_stat.csv'.format(HCOV))

        df_monsters = df_monsters.append(df_monster_hcov)   # add to a seperate DataFrame
        
    elif len(df_filter_hcov) == 1:          # just one filter = no Join/Or([]) for pkey_filter
        query_monster_hcov = And([PropertyIsEqualTo(propertyname='pkey_filter', literal=df_filter_hcov['pkey_filter'].values[0]),
                                  Or([PropertyIsEqualTo(propertyname='kationen', literal='True'), PropertyIsEqualTo(propertyname='zware_metalen', literal='True')])])
        df_monster_hcov = gwmonster.search(query=query_monster_hcov,
                                           return_fields=('pkey_filter', 'datum_monstername', 'x', 'y', 'parameter', 'waarde', 'eenheid'),
                                           max_features=9999)  

        df_monster_hcov['aquifer'] = HCOV  # geef de HCOV mee aan de monster opnames.
        # df_monster_hcov.to_csv('HCOV_{}_monsters.csv'.format(HCOV))

        # Fe
        df_Fe = df_monster_hcov.loc[df_monster_hcov['parameter'] == ('Fe')]
        # df_Fe.to_csv('Fe_HCOV_{}.csv'.format(HCOV))
        if not df_Fe.empty:
            df_Fe_stat = df_Fe.groupby(['pkey_filter', 'x', 'y']).describe().waarde
            # df_Fe_stat.to_csv('Fe_HCOV_{}_stat.csv'.format(HCOV))

        # As
        df_As = df_monster_hcov.loc[df_monster_hcov['parameter'] == ('As')]
        # df_As.to_csv('As_HCOV_{}.csv'.format(HCOV))
        if not df_As.empty:    
            df_As_stat = df_As.groupby(['pkey_filter', 'x', 'y']).describe().waarde
            # df_As_stat.to_csv('As_HCOV_{}_stat.csv'.format(HCOV))

        df_monsters = df_monsters.append(df_monster_hcov)  # add to a seperate DataFrame
            
    else:
        empty_hcov.append(HCOV)         # list with HCOVs with no filters


[000/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[200/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[250/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[300/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[350/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[400/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[450/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[500/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[550/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[600/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[650/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[700/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[750/1884] cccccccccccccccccccccccccccccccccccccccccccccccccc
[800/188

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


[000/004] cccc
[000/108] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/108] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/108] cccccccc
[000/008] cccccccc
[000/230] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/230] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/230] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/230] cccccccccccccccccccccccccccccccccccccccccccccccccc
[200/230] cccccccccccccccccccccccccccccc
[000/007] ccccccc
[000/122] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/122] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/122] cccccccccccccccccccccc
[000/006] cccccc
[000/166] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/166] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/166] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/166] cccccccccccccccc
[000/001] c
[000/030] cccccccccccccccccccccccccccccc
[000/020] cccccccccccccccccccc
[000/529] cccccccccccccccccccccccccccccccccccccccccccccc

In [43]:
print('query done, total number of sample features requested:', (len(df_monsters)))

query done, total number of sample features requested: 3241714


In [44]:
print('HCOV units with no filters and/or Fe/As measurements:', empty_hcov)

HCOV units with no filters and/or Fe/As measurements: ['0110', '0130', '0132', '0173', '0212', '0222', '0240', '0241', '0255', '0432', '0433', '0434', '0436', '0442', '0443', '0630', '0925', '1011', '0121', '0122', '0123', '1033', '1034', '1035', '1130', '1140', '1150', '1200', '1210', '1220', '1230']


A lot of data is being requested with this code, at the time of writing the maximum amount of features you can request with pydov is 10 000, this is a limitation that can be worked around but can be time-consuming. The code also requests all measurements over the whole measuring period that DOV has, so in time there will be too many features in the databank for this code... A possible way to solve this is by requesting data per year or season. 